In [1]:
%pylab inline
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
plt.style.use('ggplot')
dataq1 = pd.read_csv('2017Q1-capitalbikeshare-tripdata.csv')
dataq2 = pd.read_csv('2017Q2-capitalbikeshare-tripdata.csv')
dataq3 = pd.read_csv('2017Q3-capitalbikeshare-tripdata.csv')
dataq4 = pd.read_csv('2017Q4-capitalbikeshare-tripdata.csv')

Populating the interactive namespace from numpy and matplotlib


In [2]:
data_conc = pd.concat([dataq1, dataq2, dataq3, dataq4], ignore_index=True)

In [3]:
df = pd.DataFrame(data_conc)

In [4]:
df.head()

,Duration,Startdate,Hour,Enddate,Day,Season,Startstationnumber,Startstation,Endstationnumber,Endstation,Bikenumber,Membertype
0,221,1/1/2017 0:00,0,1/1/2017 0:04,Sunday,Spring,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member
1,1676,1/1/2017 0:06,0,1/1/2017 0:34,Sunday,Spring,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual
2,1356,1/1/2017 0:07,0,1/1/2017 0:29,Sunday,Spring,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual
3,1327,1/1/2017 0:07,0,1/1/2017 0:29,Sunday,Spring,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual
4,1636,1/1/2017 0:07,0,1/1/2017 0:34,Sunday,Spring,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual


In [5]:
df.shape

(3558924, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558924 entries, 0 to 3558923
Data columns (total 12 columns):
Duration              int64
Startdate             object
Hour                  int64
Enddate               object
Day                   object
Season                object
Startstationnumber    int64
Startstation          object
Endstationnumber      int64
Endstation            object
Bikenumber            object
Membertype            object
dtypes: int64(4), object(8)
memory usage: 325.8+ MB


In [7]:
df.isnull().sum()

Duration              0
Startdate             0
Hour                  0
Enddate               0
Day                   0
Season                0
Startstationnumber    0
Startstation          0
Endstationnumber      0
Endstation            0
Bikenumber            0
Membertype            0
dtype: int64

In [13]:
df1 = pd.DataFrame(df.groupby(['Hour','Season','Day','Membertype']).count()['Duration']).reset_index()

In [14]:
df1

,Hour,Season,Day,Membertype,Duration
0,0,Fall,Friday,Casual,333
1,0,Fall,Friday,Member,730
2,0,Fall,Monday,Casual,488
3,0,Fall,Monday,Member,502
4,0,Fall,Saturday,Casual,621
...,...,...,...,...,...
1339,23,Winter,Thursday,Member,1274
1340,23,Winter,Tuesday,Casual,193
1341,23,Winter,Tuesday,Member,1074
1342,23,Winter,Wednesday,Casual,246


In [15]:
df2 = df1.rename(columns = {'Duration':'Rider Count'})

In [16]:
df2.head()

,Hour,Season,Day,Membertype,Rider Count
0,0,Fall,Friday,Casual,333
1,0,Fall,Friday,Member,730
2,0,Fall,Monday,Casual,488
3,0,Fall,Monday,Member,502
4,0,Fall,Saturday,Casual,621


In [17]:
for column in ['Season', 'Day','Membertype']:
    df2[column]=df2[column].astype("category")

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 5 columns):
Hour           1344 non-null int64
Season         1344 non-null category
Day            1344 non-null category
Membertype     1344 non-null category
Rider Count    1344 non-null int64
dtypes: category(3), int64(2)
memory usage: 25.7 KB


In [19]:
df['Season'].unique()

array(['Spring', 'Summer', 'Fall', 'Winter'], dtype=object)

In [20]:
df['Day'].unique()

array(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday'], dtype=object)

In [86]:
df['Membertype'].unique()

array(['Member', 'Casual'], dtype=object)

In [21]:
mapping = {"Spring": 1, "Summer":2, "Fall":3, "Winter": 4}
df2['Season'] = df2['Season'].replace(mapping).astype(np.float64)

In [22]:
mapping2 = {"Monday": 1, "Tuesday":2, "Wednesday":3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7}
df2['Day'] = df2['Day'].replace(mapping2).astype(np.float64)

In [23]:
mapping3 = {"Member": 1, "Casual":0}
df2['Membertype'] = df2['Membertype'].replace(mapping3).astype(np.float64)

In [25]:
#independent variables
X=df2.drop('Rider Count',axis=1)
#dependent variable
y=df2['Rider Count']

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [32]:
from sklearn.linear_model import LinearRegression
linearreg=LinearRegression()
linearreg.fit(X_train,y_train)
predictions=linearreg.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test,predictions)

0.2995057380289555

In [31]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)
print("The accuracy of Decision Tree Regressor is: ", clf.score(X_test, y_test) )

The accuracy of Decision Tree Regressor is:  0.9686934586929985
